## Settings

In [1]:
need_to_encode = True # if you need to clean and encode texts

## Read model

In [2]:
import encoder

if (need_to_encode):
    model = encoder.parse_model('models/ruwikiruscorpora_upos_skipgram_300_2_2018.vec')
    
    print(len(model))

384764


## Clean train texts

In [3]:
import pickle

if (need_to_encode):
    texts = encoder.load_from_pickle('data/train_texts_cleaned_short.pickle')
    print(f'{len(texts)} texsts are loaded')

    cleaned_texts = list(map(lambda t: encoder.clean_text(t, model), texts[:100]))
    print(f'{len(cleaned_texts)} texts are cleaned')
    print(cleaned_texts[1])

4492 texsts are loaded


/home/yordwynn/repos/urban-waffle/encoder.py:59: FutureWarning: Possible nested set at position 30
  text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)


100 texts are cleaned
досье_NOUN роман_NOUN досье_NOUN егоров_NOUN перевод_NOUN палимпсест_NOUN издательство_NOUN оформление_NOUN начаться_VERB июльский_ADJ поехать_VERB отпуск_NOUN тогдашний_ADJ подруга_NOUN брюнетка_NOUN грозный_NOUN характер_NOUN отец_NOUN согласиться_VERB дать_VERB машина_NOUN новенький_ADJ рено_NOUN удовольствие_NOUN пролетать_VERB километр_NOUN национальный_ADJ автострада_NOUN департамент_NOUN запад_NOUN франция_NOUN побережье_NOUN атлантический_ADJ океан_NOUN далее_ADV прима_NOUN веда_NOUN серия_NOUN ехать_VERB слегка_ADV превышать_VERB скорость_NOUN рисковать_VERB прямая_NOUN линия_NOUN зелёный_ADJ свет_NOUN светофор_NOUN гарантировать_VERB полный_ADJ безопасность_NOUN момент_NOUN приближаться_VERB светофор_NOUN наперерез_ADV выскочить_VERB фольксваген_NOUN резко_ADV вывернуть_VERB руль_NOUN избежать_VERB столкновение_NOUN получиться_VERB потерять_VERB управление_NOUN рено_NOUN вылететь_VERB шоссе_NOUN пробить_VERB ограждение_NOUN скатиться_VERB крутой_ADJ отко

## Encode texts to vectors

In [4]:
if (need_to_encode):
    encoded = list(map(lambda t: encoder.encode_text(t, model, (256, 300)), cleaned_texts))

    print(f'{len(encoded)} texts are encoded')
    print(encoded[1])

    encoder.save_to_pickle(encoded, 'data/train_texts_cleaned_short_encoded.pickle')

100 texts are encoded
[[ 0.056797  0.077029  0.041735 ... -0.065106 -0.021656 -0.030767]
 [ 0.031343  0.034474  0.053584 ... -0.02193   0.020418 -0.079924]
 [ 0.056797  0.077029  0.041735 ... -0.065106 -0.021656 -0.030767]
 ...
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]]


## Load encoded data

In [5]:
labels = encoder.load_from_pickle('data/train_labels2.pickle')
encoded = encoder.load_from_pickle('data/train_texts_cleaned_short_encoded.pickle')

print(f'{len(labels)} labels are loaded')
print(f'{len(encoded)} encoded texts are loaded')
print(encoded[1])

4492 labels are loaded
100 encoded texts are loaded
[[ 0.056797  0.077029  0.041735 ... -0.065106 -0.021656 -0.030767]
 [ 0.031343  0.034474  0.053584 ... -0.02193   0.020418 -0.079924]
 [ 0.056797  0.077029  0.041735 ... -0.065106 -0.021656 -0.030767]
 ...
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]]


## Convert load data to PyTorch

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader

tensors_x = torch.FloatTensor(encoded)
tensors_y = torch.FloatTensor(labels[:100])

dataset = TensorDataset(tensors_x, tensors_y)

print(f'{len(dataset)} encoded texts are converted to tensors')
print(dataset[1])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
100 encoded texts are converted to tensors
(tensor([[ 0.0568,  0.0770,  0.0417,  ..., -0.0651, -0.0217, -0.0308],
        [ 0.0313,  0.0345,  0.0536,  ..., -0.0219,  0.0204, -0.0799],
        [ 0.0568,  0.0770,  0.0417,  ..., -0.0651, -0.0217, -0.0308],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]), tensor(1.))


## Split data to train and val

In [15]:
from torch.utils.data import random_split

#I took train and val sets in a ratio of 7 to 3
train, val = random_split(dataset, [70, 30])

train_loader = DataLoader(train, batch_size=100)
val_loader = DataLoader(val, batch_size=100)

train set: 1
val set: 1
